In [14]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold

# 저장된 모델을 복원
from tensorflow.keras.models import load_model

# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)


In [2]:
# 랜덤 시드 설정
np.random.seed(1)
tf.random.set_seed(1)

In [3]:
df1 = pd.read_csv('data/wine.csv', header=None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [4]:
df1.info() # 문자열 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6497 non-null   float64
 1   1       6497 non-null   float64
 2   2       6497 non-null   float64
 3   3       6497 non-null   float64
 4   4       6497 non-null   float64
 5   5       6497 non-null   float64
 6   6       6497 non-null   float64
 7   7       6497 non-null   float64
 8   8       6497 non-null   float64
 9   9       6497 non-null   float64
 10  10      6497 non-null   float64
 11  11      6497 non-null   int64  
 12  12      6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [6]:
df1.isna().sum() # 결측치 확인

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64

In [7]:
# 입력과 결과로 나눈다.
X = df1.drop(12, axis=1)
y = df1[12]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [8]:
# 데이터 표준화 
scaler1 = StandardScaler()
scaler1.fit(X)
X = scaler1.transform(X)
X

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  0.19309677,
        -0.91546416, -0.93722961],
       [ 0.45103572,  3.28223494, -2.19283252, ...,  0.99957862,
        -0.58006813, -0.93722961],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.79795816,
        -0.58006813, -0.93722961],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ..., -0.47897144,
        -0.91546416,  0.20799905],
       [-1.32319841, -0.30169391, -0.12823371, ..., -1.016626  ,
         1.9354021 ,  1.35322771],
       [-0.93749534, -0.78765037,  0.42232597, ..., -1.41986693,
         1.09691202,  0.20799905]])

In [9]:
# 모델 설정
model = Sequential()

a1 = X.shape[1]
model.add(Dense(30, input_dim=a1))
model.add(Activation('relu'))

model.add(Dense(12))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))


In [10]:
# 학습 모델을 생성한다
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [11]:
# 학습한다.

model.fit(X, y, epochs=200, batch_size=100)


Epoch 1/200
65/65 [==============================] - 2s 5ms/step - loss: 0.5226 - accuracy: 0.8387
Epoch 2/200
65/65 [==============================] - 0s 4ms/step - loss: 0.1507 - accuracy: 0.9769
Epoch 3/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0536 - accuracy: 0.9877
Epoch 4/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0371 - accuracy: 0.9906
Epoch 5/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0308 - accuracy: 0.9935
Epoch 6/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0273 - accuracy: 0.9943
Epoch 7/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0250 - accuracy: 0.9954
Epoch 8/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0231 - accuracy: 0.9955
Epoch 9/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0219 - accuracy: 0.9958
Epoch 10/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0205 - accuracy: 0.9962
Epoch 11/

65/65 [==============================] - 0s 4ms/step - loss: 0.0034 - accuracy: 0.9989
Epoch 83/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.9989
Epoch 84/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0031 - accuracy: 0.9991
Epoch 85/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 0.9991
Epoch 86/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 0.9991
Epoch 87/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 0.9991
Epoch 88/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 0.9992
Epoch 89/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0024 - accuracy: 0.9992
Epoch 90/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0026 - accuracy: 0.9991
Epoch 91/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0061 - accuracy: 0.9982
Epoch 92/200


65/65 [==============================] - 0s 4ms/step - loss: 7.7076e-04 - accuracy: 0.9997
Epoch 161/200
65/65 [==============================] - 0s 4ms/step - loss: 6.5910e-04 - accuracy: 0.9995
Epoch 162/200
65/65 [==============================] - 0s 4ms/step - loss: 7.9681e-04 - accuracy: 0.9995
Epoch 163/200
65/65 [==============================] - 0s 4ms/step - loss: 6.3956e-04 - accuracy: 0.9995
Epoch 164/200
65/65 [==============================] - 0s 4ms/step - loss: 6.8317e-04 - accuracy: 0.9995
Epoch 165/200
65/65 [==============================] - 0s 4ms/step - loss: 9.5125e-04 - accuracy: 0.9992
Epoch 166/200
65/65 [==============================] - 0s 5ms/step - loss: 6.3144e-04 - accuracy: 0.9997
Epoch 167/200
65/65 [==============================] - 0s 4ms/step - loss: 9.8590e-04 - accuracy: 0.9995
Epoch 168/200
65/65 [==============================] - 0s 4ms/step - loss: 7.6403e-04 - accuracy: 0.9997
Epoch 169/200
65/65 [==============================] - 0s 4ms/step - 

In [12]:
# 평가
a1 = model.evaluate(X, y)
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')


204/204 [==============================] - 1s 3ms/step - loss: 6.1082e-04 - accuracy: 0.9995
손실률 : 0.0006108235684223473
정확도 : 0.999538242816925


In [13]:
# 모델 저장
model.save('model/16/model.h5')

In [15]:
# 모델을 복원한다.
model2 = load_model('model/16/model.h5')
model2


In [16]:
a1 = model2.evaluate(X, y) # 저장한 파일이 맞는지 확인
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')


204/204 [==============================] - 1s 3ms/step - loss: 6.1082e-04 - accuracy: 0.9995
손실률 : 0.0006108235684223473
정확도 : 0.999538242816925
